In [2]:
import torch

In [3]:
world_size = torch.cuda.device_count() 
world_size

1

In [4]:
world_size = torch.cuda.device_count() if torch.cuda.is_available() else 1
print(f"World size: {world_size}")

World size: 1


/home/hindupur.v/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

/home/hindupur.v/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from torchvision.models import resnet50

In [6]:
# Define a function to initialize the process group
def setup(rank, world_size, backend='gloo'):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group(backend, rank=rank, world_size=world_size)

# Cleanup function to destroy the process group
def cleanup():
    dist.destroy_process_group()

In [7]:
# Example training function
def train(rank, world_size):
    print(f"Running basic DDP example on rank {rank}.")

    # Initialize the process group
    setup(rank, world_size, backend='gloo')

    # Load the ResNet-50 model
    model = resnet50(pretrained=True)

    # Replace the final layer for binary classification
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)

    # Wrap the model with DDP
    model = DDP(model, device_ids=None)  # No device_ids needed for CPU

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)

    # Dummy training loop
    for epoch in range(2):  # Just a few epochs for demonstration
        optimizer.zero_grad()
        # Dummy inputs and labels
        inputs = torch.randn(5, 3, 224, 224)
        labels = torch.randint(0, 2, (5,))
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        print(f"Rank {rank}, Epoch {epoch}, Loss: {loss.item()}")

    # Cleanup the process group
    cleanup()

In [8]:
if __name__ == "__main__":
    world_size = 2  # Example: using 2 processes for demonstration
    mp.spawn(train,
             args=(world_size,),
             nprocs=world_size,
             join=True)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/shared/centos7/anaconda3/2022.05/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
  File "/shared/centos7/anaconda3/2022.05/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/shared/centos7/anaconda3/2022.05/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    exitcode = _main(fd, parent_sentinel)
  File "/shared/centos7/anaconda3/2022.05/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'train' on <module '__main__' (built-in)>
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'train' on <module '__main__' (built-in)>


ProcessExitedException: process 1 terminated with exit code 1

Without parallelization

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import os

In [10]:
# Data transformations for pre-processing
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [12]:
# Load your datasets

data_dir = '/home/hindupur.v/varsha_hpp/dataset/lung_colon_image_set'

train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'valid'), transform=transform)

In [13]:
# DataLoaders

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [16]:
# Initialize ResNet-50 model
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features

num_classes=5
# Adjust for your number of classes
model.fc = nn.Linear(num_ftrs, num_classes)

In [17]:
# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [22]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [ ]:
# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    total_samples = 0 # keeps track of the total number of samples processed in the current epoch
    correct_predictions = 0 # counts the number of correctly predicted labels
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_samples
    
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.nn.parallel import DistributedDataParallel as DDP

In [8]:
class Multiclass_Classifier(nn.Module):
    def __init__(self):
        super(Multiclass_Classifier, self).__init__()
        self.name = "CNN3_ColonClassifier"
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 9, 3),
            nn.Conv2d(9, 16, 3),
            nn.MaxPool2d(2),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 5),
            nn.Conv2d(32, 32, 3),
            nn.MaxPool2d(2),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, 3),
            nn.Conv2d(64, 64, 3),
            nn.MaxPool2d(2),
            nn.ReLU()
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(64, 128, 3),
            nn.Conv2d(128, 128, 3),
            nn.MaxPool2d(2),
            nn.ReLU()
        )
        self.fc1 = nn.Linear(128 * 3 * 3, 512)  # Adjusted size after convolutional layers
        self.fc2 = nn.Linear(512, 128)  # Adjusted size
        self.fc3 = nn.Linear(128, 5)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(-1, 128 * 3 * 3)  # Adjusted size
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = x.squeeze(1)
        return F.log_softmax(x, dim=1)

In [9]:
def main():
    # Determine the world size
    world_size = torch.cuda.device_count() if torch.cuda.is_available() else 1
    print(f"World size: {world_size}")

    # Set up training data
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset, num_replicas=world_size, rank=0)
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=(train_sampler is None), sampler=train_sampler)

    # Initialize model
    model = Multiclass_Classifier()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Wrap the model with DistributedDataParallel
    if world_size > 1:
        model = DDP(model)

    # Define loss function and optimizer
    criterion = nn.NLLLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    # Training loop
    for epoch in range(5):  # Adjust the number of epochs as needed
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            outputs = model(inputs)
            print(outputs.shape, labels.shape)
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

if __name__ == "__main__":
    main()

World size: 1
Files already downloaded and verified


RuntimeError: Calculated padded input size per channel: (2 x 2). Kernel size: (3 x 3). Kernel size can't be greater than actual input size

In [13]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from torchvision.models import resnet50

In [11]:
# Check for available device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [12]:
n_gpus = torch.cuda.device_count()
n_gpus

1

In [14]:
def setup(rank, world_size, backend='gloo'):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group(backend, rank=rank, world_size=world_size)

In [15]:
def cleanup():
    dist.destroy_process_group()


In [17]:
def train(rank, world_size):
    print(f"Running on rank {rank}.")
    setup(rank, world_size)

    # Model
    model = resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 5)  # Assuming binary classification
    model = model.to(rank)
    ddp_model = DDP(model, device_ids=[rank])

    # Dummy data and training loop for demonstration
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss().to(rank)
    for epoch in range(2):  # Simple loop for demonstration
        optimizer.zero_grad()
        outputs = ddp_model(torch.randn(20, 3, 224, 224).to(rank))
        labels = torch.randint(0, 2, (20,)).to(rank)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        print(f"Rank {rank}, Epoch {epoch}, Loss: {loss.item()}")

    cleanup()

In [19]:
if __name__ == "__main__":
    world_size = torch.cuda.device_count() if torch.cuda.is_available() else 1
    print(f"World size: {world_size}")

    if world_size == 1:
        # Adjusting for single GPU or CPU, use multiprocessing to simulate DDP
        size = 4  # Simulating 4 processes for demonstration
        mp.set_start_method("spawn")
        processes = []
        for rank in range(size):
            p = mp.Process(target=setup, args=(rank, size, train))
            p.start()
            processes.append(p)

        for p in processes:
            p.join()
    else:
        # Assuming GPU environment, directly call the training function for DDP
        # This part requires running the script with torch.distributed.launch or torchrun for multi-GPU setup
        rank = 0  # This will be set properly by the torchrun utility in real execution
        train(rank, world_size)

World size: 1


RuntimeError: context has already been set

In [ ]:
srun --partition=gpu --nodes=1 --gres=gpu:v100-sxm2:1 --cpus-per-task=2 --mem=10GB --time=02:00:00 --pty /bin/bash
module load anaconda3/2022.05 cuda/11.7
conda create --name pytorch_env python=3.9 -y
source activate pytorch_env
conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia -y
python -c 'import torch; print(torch.cuda.is_available())'